In [15]:
!pip install selenium

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
!apt-get update && apt-get install -y chromium-chromedriver

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.8 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [666 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2868 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1439 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2430 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [699 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [34.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2220 kB]

In [25]:
import sys
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
from tqdm import tqdm_notebook
from datetime import datetime
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings

warnings.filterwarnings('ignore')

class SeleniumCrawler(object):
    def crawling_blog_urls(self, keyword, how_many_years, save_YN='Y'):
        yyyymmdd_list = [str(datetime.today() - relativedelta(years=i))[:10] for i in range(how_many_years)]

        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('lang=ko_KR')  # KR 언어
        
        if sys.platform == 'linux':
            driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=options)
        else:
            driver = webdriver.Chrome('./chromedriver', options=options)

        for yyyymmdd_i in range(len(yyyymmdd_list) - 1):
            # ----- 시작점 & 끝점 정하기
            yyyymmdd_from = yyyymmdd_list[yyyymmdd_i + 1]
            yyyymmdd_to = yyyymmdd_list[yyyymmdd_i]
            print('==============================================')
            print('From {} to {}'.format(yyyymmdd_from, yyyymmdd_to))

            search_url = "https://search.naver.com/search.naver?where=blog&query={}&sm=tab_opt&nso=so%3Add%2Cp%3Afrom{}to{}"

            #     driver.get("https://search.naver.com/search.naver?where=blog&query={}&sm=tab_opt&nso_open=1&nso=so:dd,p:1y,a:all".format(keyword))
            driver.get(search_url.format(keyword,
                                         str(yyyymmdd_from)[:10].replace('-', ''),
                                         str(yyyymmdd_to)[:10].replace('-', '')))

            body = driver.find_element_by_css_selector('body')
            # -----PageDown을 위해 click 추가
            body.click()

            #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            for j in range(200):
                time.sleep(0.2)
                body.send_keys(Keys.PAGE_DOWN)

            # -----내용 담긴 것 가져오기
            i = 0
            while True:
                page_info = driver.find_elements_by_class_name('api_subject_bx')[i].find_elements_by_tag_name('li')

                if len(page_info) == 0:
                    i += 1
                    continue
                else:
                    break

            # -----결과 담을 딕셔너리
            result_blog_dict = dict()

            for i, info in enumerate(page_info):
                # -----글 제목은 url가져올때 가져오는 게 편함
                try:
                    url = \
                    info.find_element_by_class_name('total_area').get_attribute('innerHTML').split('data-url="')[1].split(
                        '"')[0]
                    date = \
                    info.find_element_by_class_name('total_area').find_element_by_class_name('total_sub').text.split('\n')[
                        0]
                    name = \
                    info.find_element_by_class_name('total_area').find_element_by_class_name('total_sub').text.split('\n')[
                        1]
                    title = info.find_element_by_xpath('//*[@id="sp_blog_{}"]/div/div/a'.format(i + 1)).text
                    result_blog_dict[i] = [date, name, title, url]
                except:
                    print('All page has been collected!')
                    break

            # -----담긴 결과 pd.DataFrame으로
            if yyyymmdd_i == 0:
                result_blog = pd.DataFrame(result_blog_dict).transpose()
                result_blog.columns = ['date', 'name', 'title', 'url']
            else:
                temp = pd.DataFrame(result_blog_dict).transpose()
                temp.columns = ['date', 'name', 'title', 'url']
                result_blog = pd.concat([result_blog, temp], axis=0).reset_index(drop=True)

        #     #-----hospital_filter로 필터링
        #     if hospital_filter!='None':
        #         result_blog = result_blog.loc[result_blog['name'].map(lambda x: hospital_filter in x)].reset_index(drop=True)

        # -----드라이버 종료
        driver.quit()

        # -----url 내에 불순물 제거
        try:
            result_blog['url'] = result_blog['url'].map(lambda x: x.replace('?Redirect=Log&amp;logNo=', '/'))
        except:
            print("There's no url")

        keyword_save = keyword.replace('"', '')

        if save_YN == 'N':
            return result_blog
        else:
            result_blog.to_csv('blog_urls_{}_from{}_to{}.csv'.format(
                '+'.join(keyword_save.split(' ')),
                str(yyyymmdd_list[how_many_years - 1])[:10],
                str(yyyymmdd_list[0])[:10]
            ),
                encoding='utf-8-sig')
            return result_blog


    def crawling_blog_contents(self, result_blog, keyword, how_many_years, save_YN='Y'):
        yyyymmdd_list = [str(datetime.today() - relativedelta(years=i))[:10] for i in range(how_many_years)]

        # 결과 담을 딕셔너리
        result_url = dict()

        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('lang=ko_KR')  # KR 언어

        if sys.platform == 'linux':
            driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=options)
        else:
            driver = webdriver.Chrome('./chromedriver', options=options)

        # url 돌면서 내용 수집
        for i, url in enumerate(result_blog['url']):
            try:
                driver.get(url)
            except:
                print('Error happend at {}; Time out'.format(i))
                temp = 'deleted'
            # driver.get(result['url'][1])

            # -----inner frame 진입
            try:
                driver.switch_to.frame('mainFrame')
            except:
                print('Error happend at {}; content deleted'.format(i))

            # -----contents 받아오기
            try:
                temp = driver.find_element_by_xpath('//*[@id="post-view{}"]/div/div[3]'.format(url.split('/')[-1])).text
                # temp = driver.find_element_by_xpath('//*[@id="post-view{}"]/div/div[3]'.format(result['url'][1].split('/')[-1])).text
            except:
                temp = 'deleted'

            # -----담기
            result_url[i] = temp

        driver.close()

        # -----기존 결과에 cbind
        result_url = pd.Series(result_url).rename('content')
        result_contents = pd.concat([result_blog, result_url], axis=1)

        keyword_save = keyword.replace('"', '')

        if save_YN == 'N':
            return result_contents
        else:
            result_contents.to_csv('blog_contents_{}_from{}_to{}.csv'.format(
                '+'.join(keyword_save.split(' ')),
                str(yyyymmdd_list[how_many_years - 1])[:10],
                str(yyyymmdd_list[0])[:10]
            ),
                encoding='utf-8-sig')
            return result_contents


    def crawling_blog_total(self, keyword, hospital_filter, save_YN='Y'):
        # 앞의 함수 실행시켜서 결과 담아두기
        result_final = self.crawling_blog_urls(keyword, hospital_filter)

        # 결과 담을 딕셔너리
        result_url = dict()

        options = webdriver.ChromeOptions()
        # options.add_argument('headless')
        options.add_argument('lang=ko_KR')  # KR 언어

        if sys.platform == 'linux':
            driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=options)
        else:
            driver = webdriver.Chrome('./chromedriver', options=options)

        # url 돌면서 내용 수집
        for i, url in tqdm_notebook(enumerate(result_final['url'])):
            driver.get(url)
            # driver.get(result['url'][1])

            # -----inner frame 진입
            try:
                driver.switch_to.frame('mainFrame')
            except:
                print('Error happend at {}'.format(i))

            # -----contents 받아오기
            try:
                temp = driver.find_element_by_xpath('//*[@id="post-view{}"]/div/div[3]'.format(url.split('/')[-1])).text
                # temp = driver.find_element_by_xpath('//*[@id="post-view{}"]/div/div[3]'.format(result['url'][1].split('/')[-1])).text
            except:
                temp = 'deleted'

            # -----담기
            result_url[i] = temp

        driver.close()

        # -----기존 결과에 cbind
        result_url = pd.Series(result_url).rename('content')
        result_final = pd.concat([result_final, result_url], axis=1)

        if save_YN == 'N':
            return result_final
        else:
            result_final.to_csv('data/blog_contents_{}_{}.csv'.format('+'.join(keyword.split(' ')),
                                                                      str(datetime.today())[:10]),
                                encoding='utf-8-sig')
            return result_final


In [26]:
keyword_list = ['"백내장"+"부수입"',
                '"백내장"+"소개"',
                '"백내장"+"수당"',
                '"백내장"+"숙소"',
                '"백내장"+"실비"',
                '"백내장"+"실손"',
                '"백내장"+"페이백"',
                '"백내장"+"할인"',
                '"백내장"+"호텔"']

import time

sc = SeleniumCrawler()
start = time.time()
for keyword in keyword_list:
    result_blog = sc.crawling_blog_urls(keyword, 2, 'N')
    sc.crawling_blog_contents(result_blog, keyword, 2)
end = time.time() - start

print(f'elapsed = {end}')

From 2020-11-12 to 2021-11-12
From 2020-11-12 to 2021-11-12
Error happend at 99; content deleted
Error happend at 108; content deleted
Error happend at 116; content deleted
Error happend at 166; content deleted
Error happend at 183; content deleted
Error happend at 211; content deleted
Error happend at 241; content deleted
Error happend at 271; content deleted
Error happend at 293; content deleted
Error happend at 329; content deleted
Error happend at 331; content deleted
Error happend at 386; content deleted
Error happend at 399; content deleted
Error happend at 439; content deleted
Error happend at 453; content deleted
Error happend at 456; content deleted
Error happend at 491; content deleted
Error happend at 493; content deleted
Error happend at 510; content deleted
Error happend at 591; content deleted
Error happend at 632; content deleted
Error happend at 635; content deleted
Error happend at 637; content deleted
Error happend at 694; content deleted
Error happend at 699; content